In [2]:
pip install nltk

     |████████████████████████████████| 1.4MB 6.6MB/s eta 0:00:01
     |████████████████████████████████| 307kB 21.3MB/s eta 0:00:01
     |████████████████████████████████| 675kB 11.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 13.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import numpy as np
import re
import nltk
import ast
from nltk import word_tokenize, pos_tag, pos_tag_sents

In [4]:
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger') 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords


from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet 
# Create WordNetLemmatizer object 


lemmatizer = WordNetLemmatizer() 

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [5]:
df = pd.read_csv('opinion_survey.csv')
df = df.replace(np.nan, 'Neutral', regex=True) ### Some values are missing, use "Neutral" for missing values
df = df.drop(df.columns[[0]],axis=1) ### Drop extra index
test_df = df.iloc[0:, 0:] # Just for renaming purposes so that df variable can be used for testing
test_df

,I can easily log-in and log-out my Canvas account. (Please write your views relating your answer above.),I can access my course using the mobile Canvas app. (Please write your views about this relating to your answer above.),I can access my Canvas account using the available computers in JRU. (Please write your views about this relating to your answer above.),I have no issues accessing my Canvas account using the wired internet connection in the computer laboratories. (Please write your views about this relating to your answer above.),I can easily get help when I have problems with my Canvas account. (Please write your views about this relating to your answer above.),I can access my Canvas account outside JRU. (Please write your views about this relating to your answer above.),I can clearly understand the lessons on my Canvas modules. (Please write your views about this relating to your answer above.),I recognize that the modules in my course are well organized. (Please write your views about this relating to your answer above.),I can easily manage the navigation of the courses through the desktop version. .1,I can easily manage the navigation of the courses through the mobile app. (Please write your views about this relating to your answer above.),...,I understand the lessons better with the help of course materials in Canvas. (Please write your views about this relating to your answer above.),Canvas course materials make me interested. (Please write your views about this relating to your answer above.),"The course materials on Canvas helped me prepare for: Face to face discussions, quizzes, major exams, assignments, forums, etc. (Please write your views about this relating to your answer above.)",My instructor knows how to use Canvas. (Please write your views about this relating to your answer above.),My instructor demonstrated mastery of the subject matter. (Please write your views about this relating to your answer above.),My instructor encourages me to use Canvas. (Please write your views about this relating to your answer above.),"Canvas has made it easy for me to: communicate with my instructor, receive class announcements, share ideas on group discussions","I can answer the assessment materials (quiz, exam) posted in Canvas. (Please write your views about this relating to your answer above.)","I find the course materials on Canvas aligned with the: quizzes, major exams, assignments, forums, etc.) (Please write your views about this relating to your answer above.)",I have been give enough time to accomplish the activities on Canvas. (Please write your views about this relating to your answer above.)
0,i can,i can,i can,i can,i can,i can,i can,i can,i can,i can,...,i can,i can,i can,i can,i can,i can,i can,i can,i can,-
1,i have no problems in log in,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,...,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem,i have no problem
2,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,...,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,I have no problems with it,"Yes, I have no problems with it"
3,i dont have problem in my canvas,i can easily login my canvas app in my mobile app,I can access my Canvas account using the avail...,i dont have problem in internet connection,i dont have problem with it,i dont have problem with it,i dont have problem with it,i dont have problem with it,i d

In [6]:
stoplist = stopwords.words('english') + ['though']
def remove_stopWords(w): 
    w = ' '.join(word for word in w.split() if word not in stoplist)
    return w

In [18]:
## Reduce list dimension

def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [30]:
col_range = len(test_df.columns) # number of columns
test_df = test_df.replace(r'^\s*$', "neutral", regex=True) ## If row value is null, replace with neutral string
aspect_list = []

for i in range(0,col_range):
    col = test_df.columns[i] # The current column
    test_df.loc[:,col] = test_df[col].apply(lambda x : str.lower(str(x))) ## To Lower Case
    test_df.loc[:,col] = test_df[col].apply(lambda x : " ".join(re.findall('[\w]+',x))) # Remove Punctuations
    test_df.loc[:,col] = test_df[col].apply(lambda x : remove_stopWords(x)) # Remove Stop words
    
    ##POS TAGGING
    texts = test_df.loc[:,col].tolist()
    tagged_texts = pos_tag_sents(map(word_tokenize, texts)) ### Tag every word in a row with POS
    
    #inputTupples=ast.literal_eval(tagged_texts)
    flattened_tagged = flatten_list(tagged_texts)
    prevWord=''
    prevTag=''
    currWord=''
    aspectList=[]
    outputDict={}
    
    #Extracting Aspects
    
    for word,tag in flattened_tagged:
        if(tag=='NN' or tag=='NNP'):
            if(prevTag=='NN' or prevTag=='NNP'):
                currWord= prevWord + ' ' + word
            else:
                aspectList.append(prevWord.upper())
                currWord= word

        prevWord=currWord
        prevTag=tag
    #Eliminating aspect which has 1 or less count
    for aspect in aspectList:
            if(aspectList.count(aspect)>1):
                    if(outputDict.keys()!=aspect):
                            outputDict[aspect]=aspectList.count(aspect)
    outputAspect=sorted(outputDict.items(), key=lambda x: x[1],reverse = True)
    aspect_list.append(outputAspect)
    #print(outputAspect)
    
    


In [53]:
df_cols = test_df.columns.tolist()

for i in range(0,col_range):
    print(i+1," ",df_cols[i])
    print(aspect_list[i], "\n")

1   I can easily log-in and log-out my Canvas account. (Please write your views relating your answer above.)
[('CANVAS ACCOUNT', 10), ('LOG', 8), ('ACCOUNT', 4), ('INTERNET CONNECTION', 4), ('LOG LOG', 3), ('CONNECTION', 3), ('CANVAS', 3), ('LOG ACCOUNT', 3), ('DONT PROBLEM CANVAS', 2), ('DONT', 2), ('USE', 2), ('LOG PROBLEM', 2), ('JRU', 2), ('TIME', 2), ('ACCESS ACCOUNT', 2), ('PROBLEM', 2), ('LOGOUT', 2)] 

2   I can access my course using the mobile Canvas app. (Please write your views about this relating to your answer above.)
[('ACCESS COURSE', 17), ('CANVAS APP', 13), ('ACCESS', 11), ('CANVAS', 8), ('PHONE', 6), ('COURSE', 5), ('APP', 5), ('PROBLEM', 4), ('USE', 3), ('APP ACCESS COURSE', 2), ('BROWSER', 2), ('ACCESS CANVAS APP', 2), ('APPLICATION ACCESS COURSE', 2), ('APPLICATION', 2), ('ACCESS CANVAS', 2)] 

3   I can access my Canvas account using the available computers in JRU. (Please write your views about this relating to your answer above.)
[('ACCESS CANVAS ACCOUNT', 20),

In [35]:
clean_data = [[int(item.split()[0]) for item in row] for row in aspect_list]
df_cols = test_df.columns.tolist()
pd.DataFrame(clean_data, columns = df_cols)

AttributeError: 'tuple' object has no attribute 'split'

In [19]:
hello = flatten_list(tagged_texts)

for i,k in hello.items():
    print(i,k)

AttributeError: 'list' object has no attribute 'items'